In [1]:
import pandas as pd
from sqlalchemy import create_engine
database_path = "database/name_origins.sqlite"

In [2]:
engine = create_engine(f"sqlite:///{database_path}")
connection = engine.connect()

In [3]:
name_df = pd.read_sql("SELECT * FROM name", connection)
origin_df = pd.read_sql("SELECT * FROM origin", connection)

In [4]:
name_df.head()

,id,title,gender,origin_id,meaning,used_in,additional_info
0,1,Aaliyah,Girl,3,"Lofty, sublime, exalted",Arabic and English speaking countries,This particular spelling of the name Aliyah is...
1,2,Aaron,Boy,23,Mountain,English speaking countries,Anglicisation of the Hebrew Aharon. In the Bi...
2,3,Abagail,Girl,23,Father in rejoicing,English speaking countries,A modern respelling of Abigail.
3,4,Abbey,Both,23,Father in rejoicing,English speaking countries,"A nickname for Abigail, and also a surname fro..."
4,5,Abbie,Girl,23,Father in rejoicing,English speaking countries,Nickname for Abigail.


In [5]:
origin_df.head()

,id,title,count,slug
0,1,Afghan,1,afghan
1,2,African,16,african
2,3,Arabic,94,arabic
3,4,Aramaic,17,aramaic
4,5,Assyrian,1,assyrian


In [6]:
origin_extrated_df = origin_df.loc[origin_df["count"] >= 48]
origin_extrated_df

,id,title,count,slug
2,3,Arabic,94,arabic
14,15,English,617,english
16,17,French,116,french
17,18,Gaelic,304,gaelic
18,19,German,68,german
19,20,Germanic,229,germanic
20,21,Greek,393,greek
22,23,Hebrew,614,hebrew
26,27,Latin,484,latin
40,42,Spanish,48,spanish


In [7]:
name_df.groupby(by="used_in").count()

,id,title,gender,origin_id,meaning,additional_info
used_in,,,,,,
African and English speaking countries,9,9,9,9,8,8
"African, Arabic and English speaking countries",5,5,5,5,5,3
"African, Arabic, English and Swahili speaking countries",8,8,8,8,8,4
"African, Arabic, English, Hebrew and Swahili speaking countries",1,1,1,1,1,1
"African, Arabic, English, Hindi, Hungarian and Swahili speaking countries",1,1,1,1,1,1
"African, Arabic, English, Swahili and Russian speaking countries",1,1,1,1,1,1
"African, Bulgarian, Croatian, Czech, English, Polish, Serbian, Spanish, Swahili and Russian speaking countries",1,1,1,1,1,1
"African, English and Swahili speaking countries",6,6,6,6,5,5
"African, English, German and Swahili speaking countries",1,1,1,1,1,0


In [8]:
name_origin_split_df = name_df
name_origin_split_df['used_in'] = name_origin_split_df['used_in'].str.replace(' speaking countries', '')
name_origin_split_df['used_in'] = name_origin_split_df['used_in'].str.replace(' and', '')
name_origin_split_df['used_in'] = name_origin_split_df['used_in'].str.replace(',', '')

name_origin_split_df.groupby(by="used_in").count()

,id,title,gender,origin_id,meaning,additional_info
used_in,,,,,,
African Arabic English,5,5,5,5,5,3
African Arabic English Hebrew Swahili,1,1,1,1,1,1
African Arabic English Hindi Hungarian Swahili,1,1,1,1,1,1
African Arabic English Swahili,8,8,8,8,8,4
African Arabic English Swahili Russian,1,1,1,1,1,1
African Bulgarian Croatian Czech English Polish Serbian Spanish Swahili Russian,1,1,1,1,1,1
African English,9,9,9,9,8,8
African English German Swahili,1,1,1,1,1,0
African English Hebrew Polish,1,1,1,1,1,1


In [9]:
name_origin_split_df.head()

,id,title,gender,origin_id,meaning,used_in,additional_info
0,1,Aaliyah,Girl,3,"Lofty, sublime, exalted",Arabic English,This particular spelling of the name Aliyah is...
1,2,Aaron,Boy,23,Mountain,English,Anglicisation of the Hebrew Aharon. In the Bi...
2,3,Abagail,Girl,23,Father in rejoicing,English,A modern respelling of Abigail.
3,4,Abbey,Both,23,Father in rejoicing,English,"A nickname for Abigail, and also a surname fro..."
4,5,Abbie,Girl,23,Father in rejoicing,English,Nickname for Abigail.


In [37]:
name_origins = {}
for i, row in name_origin_split_df.iterrows():
    origins = name_origin_split_df.loc[i, "used_in"].split(' ')
    for origin in origins:
        origin_in_key = False
        for key in name_origins.keys():
            if origin == key:
                name_origins[origin] = name_origin_split_df.loc[i, "title"]
        if origin_in_key == False:
            name_origins[origin] = name_origin_split_df.loc[i, "title"]

In [35]:
name_origins

{'Arabic': 'Aaliyah',
 'English': 'Aaliyah',
 'Spanish': 'Abel',
 'Hebrew': 'Abigail',
 'Yiddish': 'Abram',
 'African': 'Ada',
 'Polish': 'Ada',
 'French': 'Adele',
 'Swahili': 'Adin',
 'Hindi': 'Aditya',
 'Italian': 'Adolfo',
 'Greek': 'Adonis',
 'German': 'Adrian',
 'Gaelic': 'Aidan',
 'Japanese': 'Akira',
 'Hawaiian': 'Alana',
 'Portuguese': 'Alberto',
 'Albanian': 'Alek',
 'Russian': 'Alek',
 'Slovak': 'Aleksandar',
 'Dutch': 'Alexander',
 'Hungarian': 'Alexander',
 'Czech': 'Alexandr',
 'Romanian': 'Alexandra',
 'Swedish': 'Alice',
 'Armenian': 'Alisa',
 'Bulgarian': 'Ana',
 'Croatian': 'Ana',
 'Serbian': 'Ana',
 'Danish': 'Anders',
 'Norwegian': 'Anders',
 'Finnish': 'Anika',
 'Indonesian': 'Anjali',
 'Catalan': 'Anna',
 'Estonian': 'Anna',
 'Urdu': 'Arnav',
 'Welsh': 'Arwen',
 'Latvian': 'Audra',
 'Lithuanian': 'Audra',
 'Turkish': 'Aydan',
 'Persian': 'Cyrus',
 'Chinese': 'Hung',
 'Vietnamese': 'Huy',
 'Icelandic': 'Kari',
 'Ukrainian': 'Oksana'}

In [28]:
i = enumerate(name_origin_split_df)
print(i)

TypeError: 'enumerate' object is not subscriptable